In [ ]:
import rospy
import roslib
import numpy as np
from referee_modules.referee_interface import GetInfoClient, RefereeBuffClient,RefereeGameStateClient,RefereeHitClient,RefereePenaltyClient,RefereeReloadClient
from icra_roboin_msgs.srv import SetGoal_2,BehaviorStyleSet,BlackboardGeneralInfo,RefereeBuff,RefereeGameState,RefereeHit,RefereePenalty,RefereeShoot,RefereeTag,RefereeReload,RefereeShootResponse,RefereeTagResponse
from nav_msgs.msg import Odometry
from std_msgs.msg import Header
from geometry_msgs.msg import Quaternion, PointStamped,PoseStamped
from tf.transformations import quaternion_from_euler, euler_from_quaternion
rospy.init_node('referee_test')

In [ ]:
class Robot:
    def __init__(self,robot_name = "",team="blue"):
        self.x = 0
        self.y = 0
        self.orientation = Quaternion()
        self.team = team
        self.robot_name = robot_name
        self.hp = 0
        self.ammo = 0
        self.real_pose_subscriber = rospy.Subscriber(self.robot_name + "/base_pose_ground_truth",Odometry,self.RealPoseCB)
        self.pose_publisher = rospy.Publisher(self.robot_name + "/referee_pose",PoseStamped,queue_size=1)
        self.shootmark_pub = rospy.Publisher(self.robot_name + "/shoot_mark",PointStamped,queue_size=1)
    def RealPoseCB(self,msg):
        self.x = msg.pose.pose.position.x
        self.y = msg.pose.pose.position.y
        self.orientation = msg.pose.pose.orientation
        pose = PoseStamped()
        pose.header.stamp = msg.header.stamp
        pose.header.frame_id = "referee_map"
        pose.pose = msg.pose.pose
        self.pose_publisher.publish(pose)

In [ ]:
class Referee1vs1:
    def __init__(self):
        self.robot0=Robot(robot_name="robot_0",team="blue")
        self.robot1=Robot(robot_name="robot_1",team="red")
        self.shoot_server_0 = rospy.Service("robot_0"+'/referee_shoot_service',RefereeShoot,self.ShootCB_0)
        self.shoot_server_1 = rospy.Service("robot_1"+'/referee_shoot_service',RefereeShoot,self.ShootCB_1)
        
        
    def ShootCB_0(self,req):
        point = PointStamped()
        point.header.stamp = rospy.Time.now()
        point.header.frame_id = "referee_map"
        point.point.x= req.x
        point.point.y= req.y
        point.point.z= 0.1
        
        self.robot0.shootmark_pub.publish(point)
        if req.x <-4 and req.y < -4:
            return RefereeShootResponse(5,True)
        elif ((self.robot1.x - req.x)**2) + ((self.robot1.y - req.y)**2) < 0.25:
            return RefereeShootResponse(1,True)
        else:
            return RefereeShootResponse(3,True)
        
        
        
    def ShootCB_1(self,req):
        point = PointStamped()
        point.header.stamp = rospy.Time.now()
        point.header.frame_id = "referee_map"
        point.point.x= 8-req.x
        point.point.y= 5-req.y
        point.point.z= 0.1
        
        self.robot1.shootmark_pub.publish(point)
        if req.x < -4 and req.y < -4:
            return RefereeShootResponse(5,True)
        elif ((self.robot0.x - (8-req.x))**2) + ((self.robot0.y - (5-req.y))**2) < 0.25:
            return RefereeShootResponse(1,True)
        else:
            return RefereeShootResponse(3,True)

In [ ]:
ref_obj = Referee1vs1()
rospy.spin()

In [ ]:
def ShootCB(req,robot_name):
    global robot_list
    global robot_index
    global shootmark_pub
    
    point = PointStamped()
    point.header.stamp = rospy.Time.now()
    point.header.frame_id = "map"
    
    print "ok"
    
    if(robot_list[robot_index[robot_name]].team == 0):
        point.point.x= req.x
        point.point.y= req.y
        point.point.z= 0.1
        shootmark_pub.publish(point)
        
        for item in robot_list:
            if ((item.x - req.x) * (item.x - req.x)) + ((item.y - req.y) * (item.y - req.y)) < 0.25:
                if(robot_list[robot_index[robot_name]].team == item.team):
                    return RefereeShootResponse(0,True)
                else:
                    return RefereeShootResponse(1,True)
                
    elif(robot_list[robot_index[robot_name]].team == 1):
        point.point.x= 8 - req.x
        point.point.y= 5 - req.y
        point.point.z= 0.1
        shootmark_pub.publish(point)
        
        for item in robot_list:
            if ((item.x - (8-req.x)) * (item.x - (8-req.x))) + ((item.y - (5-req.y)) * (item.y - (5-req.y))) < 0.25:
                if(robot_list[robot_index[robot_name]].team == item.team):
                    return RefereeShootResponse(0,True)
                else:
                    return RefereeShootResponse(1,True)
    
    return RefereeShootResponse(3,True)

In [ ]:
game_setting = [{"robot_name":"robot_0","team":0},{"robot_name": "robot_1","team":1}]
robot_list = []
server_list = []

robot_index = {}


for item in game_setting:
    robot_index[item["robot_name"]] = len(robot_list)
    robot_list.append(robot_state(robot_name = item["robot_name"],team = item["team"]))
    server_list.append(rospy.Service(item["robot_name"]+'/referee_shoot_service',RefereeShoot,ShootCB,item["robot_name"]))
    server_list.append(rospy.Service(item["robot_name"]+'/referee_tag_service',RefereeTag,TagCB,item["robot_name"]))


In [ ]:
class Robot:
    def __init__(self,robot_name = "",team=0):
        self.x = 0
        self.y = 0
        self.orientation = Quaternion()
        self.team = team
        self.robot_name = robot_name
        self.referee_object
        self.ally_x = 0
        self.ally_y = 0
        self.enemy_1_x = 0
        self.enemy_1_y = 0
        self.enemy_2_x = 0
        self.enemy_2_y = 0
        self.hp = 0
        self.ammo = 0
        self.subscriber = rospy.Subscriber(self.robot_name + "/base_pose_ground_truth",Odometry,self.CB)
        self.pose_publisher = rospy.Publisher(self.robot_name + "/referee_pose",PoseStamped,queue_size=1)
        self.shootmark_pub = rospy.Publisher(self.robot_name + "/shoot_mark",PointStamped,queue_size=1)
        self.shoot_server = rospy.Service(self.robot_name + '/referee_shoot_service',RefereeShoot,self.ShootCB)
    def CB(self,msg):
        self.x = msg.pose.pose.position.x
        self.y = msg.pose.pose.position.y
        self.orientation = msg.pose.pose.orientation
        pose = PoseStamped()
        pose.header.stamp = msg.header.stamp
        pose.header.frame_id = "map"
        pose.pose = msg.pose.pose
        self.pose_publisher.publish(pose)
    def ShootCB(self,req):
        point = PointStamped()
        point.header.stamp = rospy.Time.now()
        point.header.frame_id = "map"

        if(self.team == 0):
            point.point.x= req.x
            point.point.y= req.y
            point.point.z= 0.1
            shootmark_pub.publish(point)
            
            
            for item in [(self.ally_x,self.ally_y,"ally"),(self.enemy_1_x,self.enemy_1_y,"enemy1"),(self.enemy_2_x,self.enemy_2_y,"enemy2")]:
                if ((item[0] - req.x) * (item[0] - req.x)) + ((item[1] - req.y) * (item[1] - req.y)) < 0.25:
                    if(item[2] == "ally"):
                        return RefereeShootResponse(0,True)
                    elif(item[2] == "enemy1"):
                        return RefereeShootResponse(1,True)
                    elif(item[2] == "enemy2"):
                        return RefereeShootResponse(2,True)

        elif(self.team == 1):
            point.point.x= 8 - req.x
            point.point.y= 5 - req.y
            point.point.z= 0.1
            shootmark_pub.publish(point)
            
            
            for item in [(self.ally_x,self.ally_y,"ally"),(self.enemy_1_x,self.enemy_1_y,"enemy1"),(self.enemy_2_x,self.enemy_2_y,"enemy2")]:
                if ((item[0] - (8 - req.x)) * (item[0] - (8 - req.x))) + ((item[1] - (5 - req.y)) * (item[1] - (5 - req.y))) < 0.25:
                    if(item[2] == "ally"):
                        return RefereeShootResponse(0,True)
                    elif(item[2] == "enemy1"):
                        return RefereeShootResponse(1,True)
                    elif(item[2] == "enemy2"):
                        return RefereeShootResponse(2,True)

        return RefereeShootResponse(3,True)
        


In [ ]:
class RefereeObject:
    def __init__(self,robot_list):
        self.players = {}
        self.blue_1 = 0
        self.blue_2 = 0
        self.red_1 = 0
        self.red_2 = 0
        for item in robot_list:
            if item["team"] == 0:
                if self.blue_1 != 0 :
                    self.blue_2 = Robot(item["robot_name"],0)
                    self.players[item["robot_name"]] = "blue_2"
                else:
                    self.blue_1 = Robot(item["robot_name"],0)
                    self.players[item["robot_name"]] = "blue_1"
            if item["team"] == 1:
                if self.red_1 != 0 :
                    self.red_2 = Robot(item["robot_name"],1)
                    self.players[item["robot_name"]] = "red_2"
                else:
                    self.red_1 = Robot(item["robot_name"],1)
                    self.players[item["robot_name"]] = "red_1"
    def GetXY(self,robot_name):
        if robot_name = "blue_1":
            return self.blue_2.x, self.blue_2.y, self.red_1.x, self.red_1.y, self.red_2.x, self.red_2.y
        elif robot_name = "blue_2":
            return self.blue_1.x, self.blue_1.y, self.red_1.x, self.red_1.y, self.red_2.x, self.red_2.y
        elif robot_name = "red_1":
            return self.red_2.x, self.red_2.y, self.blue_1.x, self.blue_1.y, self.blue_2.x, self.blue_2.y
        elif robot_name = "red_2":
            return self.red_1.x, self.red_1.y, self.blue_1.x, self.blue_1.y, self.blue_2.x, self.blue_2.y
            
        
                
        

In [ ]:
game_setting = [{"robot_name":"robot_0","team":0},{"robot_name": "robot_1","team":1}]

In [ ]:
class robot:
    def __init__(self,robot_name = "",team=0):
        self.x = 0
        self.y = 0
        self.orientation = Quaternion()
        self.team = team
        self.robot_name = robot_name
        self.hp = 0
        self.ammo = 0
        self.subscriber = rospy.Subscriber(self.robot_name + "/base_pose_ground_truth",Odometry,self.CB)
        self.pose_publisher = rospy.Publisher(self.robot_name + "/referee_pose",PoseStamped,queue_size=1)
    def CB(self,msg):
        self.x = msg.pose.pose.position.x
        self.y = msg.pose.pose.position.y
        self.orientation = msg.pose.pose.orientation
        pose = PoseStamped()
        pose.header.stamp = msg.header.stamp
        pose.header.frame_id = "map"
        pose.pose = msg.pose.pose
        self.pose_publisher.publish(pose)

In [ ]:
def ShootCB(req,robot_name):
    global robot_list
    global robot_index
    global shootmark_pub
    
    point = PointStamped()
    point.header.stamp = rospy.Time.now()
    point.header.frame_id = "map"
    
    print "ok"
    
    if(robot_list[robot_index[robot_name]].team == 0):
        point.point.x= req.x
        point.point.y= req.y
        point.point.z= 0.1
        shootmark_pub.publish(point)
        
        for item in robot_list:
            if ((item.x - req.x) * (item.x - req.x)) + ((item.y - req.y) * (item.y - req.y)) < 0.25:
                if(robot_list[robot_index[robot_name]].team == item.team):
                    return RefereeShootResponse(0,True)
                else:
                    return RefereeShootResponse(1,True)
                
    elif(robot_list[robot_index[robot_name]].team == 1):
        point.point.x= 8 - req.x
        point.point.y= 5 - req.y
        point.point.z= 0.1
        shootmark_pub.publish(point)
        
        for item in robot_list:
            if ((item.x - (8-req.x)) * (item.x - (8-req.x))) + ((item.y - (5-req.y)) * (item.y - (5-req.y))) < 0.25:
                if(robot_list[robot_index[robot_name]].team == item.team):
                    return RefereeShootResponse(0,True)
                else:
                    return RefereeShootResponse(1,True)
    
    return RefereeShootResponse(3,True)

def TagCB(req,robot_name):
    global robot_list
    global robot_index
    
    if(robot_list[robot_index[robot_name]].team == 0):
        if ( ((robot_list[robot_index[robot_name]].x - req.x)*(robot_list[robot_index[robot_name]].x - req.x)) + ((robot_list[robot_index[robot_name]].y - req.y)*(robot_list[robot_index[robot_name]].y - req.y))   ) < 0.2:
            ###do something
            return RefereeTagResponse(1,True)
    elif(robot_list[robot_index[robot_name]].team == 1):
        if ( ((robot_list[robot_index[robot_name]].x - (8-req.x))*(robot_list[robot_index[robot_name]].x - (8-req.x))) + ((robot_list[robot_index[robot_name]].y - (5-req.y))*(robot_list[robot_index[robot_name]].y - (5-req.y)))   ) < 0.2:
            ###do something
            return RefereeTagResponse(1,True)            

    return RefereeShootResponse(0,True)

In [ ]:
game_setting = [{"robot_name":"robot_0","team":0},{"robot_name": "robot_1","team":1}]
robot_list = []
server_list = []
shootmark_pub = rospy.Publisher("shoot_mark",PointStamped,queue_size=1)
robot_index = {}


for item in game_setting:
    robot_index[item["robot_name"]] = len(robot_list)
    robot_list.append(robot_state(robot_name = item["robot_name"],team = item["team"]))
    server_list.append(rospy.Service(item["robot_name"]+'/referee_shoot_service',RefereeShoot,ShootCB,item["robot_name"]))
    server_list.append(rospy.Service(item["robot_name"]+'/referee_tag_service',RefereeTag,TagCB,item["robot_name"]))




In [ ]:
rospy.spin()

In [1]:
import rospy
import roslib
import numpy as np
from referee_modules.referee_interface import GetInfoClient, RefereeBuffClient,RefereeGameStateClient,RefereeHitClient,RefereePenaltyClient,RefereeReloadClient
from icra_roboin_msgs.srv import SetGoal_2,BehaviorStyleSet,BlackboardGeneralInfo,RefereeBuff,RefereeGameState,RefereeHit,RefereePenalty,RefereeShoot,RefereeTag,RefereeReload,RefereeShootResponse,RefereeTagResponse
from nav_msgs.msg import Odometry
from std_msgs.msg import Header
from geometry_msgs.msg import Quaternion, PointStamped,PoseStamped
from tf.transformations import quaternion_from_euler, euler_from_quaternion
from roborts_msgs.msg import RobotDamage

In [4]:
a = RobotDamage.

In [5]:
a

1